In [ ]:
import seaborn as sns
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
players_salary = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_salary.csv')
players_stats = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_players_stats_combined.csv')
players_edsm = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_endorsements.csv')
players_twitter = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_twitter_players.csv')
players_RPM = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_real_plus_minus.csv')
team_value = pd.read_csv('/kaggle/input/social-power-nba/nba_2017_att_val.csv')

In [ ]:
players_salary

In [ ]:
players_stats = players_stats.rename(columns={'PLAYER':'NAME'})
players_stats

In [ ]:
players_stats.info()

In [ ]:
p_s = players_stats.drop(players_stats[["POSITION","TEAM","Unnamed: 0","Rk","AGE"]],axis=1)

In [ ]:
pss = pd.merge(players_salary,p_s,how='inner',on='NAME')
pss = pss.drop(players_stats[["POSITION","TEAM","NAME"]],axis=1)

In [ ]:
pss

In [ ]:
pss.isnull().sum()

In [ ]:
p_s.fillna(p_s["3P%"].mean(),inplace=True)

In [ ]:
pss.fillna(p_s["3P%"].mean(),inplace=True)

In [ ]:
pss.isnull().sum()

In [ ]:
pss

In [ ]:
ps = pss.drop("SALARY",axis=1)
ps

In [ ]:
from sklearn.cluster import KMeans

Use K-Means to cluster the NBA players.Show the mean value of every feature of defferent clusters.

In [ ]:
km = KMeans(n_clusters=3).fit(ps)
ps['cluster'] = km.labels_
ps.sort_values('cluster')
cluster_centers = km.cluster_centers_
ps.groupby("cluster").mean()

According to the mean value showed,
* It seems that the features of the NBA players in cluster 0 have the highest mean value. 
* the stats of almost every feature in cluster 0 are better than 1,so do 1 and 2.

In [ ]:
from pandas.plotting import scatter_matrix 
centers = ps.groupby("cluster").mean().reset_index
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 10
colors = np.array(['red','green','blue','yellow'])
plt.scatter(ps["MP"],ps["WINS_RPM"],c=colors[ps["cluster"]])
plt.scatter(centers.MP, centers.WINS_RPM,linewidths=50,marker='+',s=3,c='black')
plt.xlabel("MP")
plt.ylabel("WINS_RPM")


In [ ]:
scatter_matrix(ps[['GP','MPG','ORPM','DRPM','RPM','WINS_RPM']],s=50,alpha=1,c=colors[ps["cluster"]],figsize=(20,20))

The clustering seems to be reasonable that the good players are good at everything.For example,players in cluster 0 have good datas of ORPM while their data of DRPM are more likely to be higher than that in cluster 1 and 2.  

Now I add feature "SALARY" and do the clustering.

In [ ]:
km1 = KMeans(n_clusters=3).fit(pss)
pss['cluster'] = km1.labels_
pss.sort_values('cluster')
cluster_centers = km1.cluster_centers_
pss.groupby("cluster").mean()

Someting wierd happens.The players in the cluster which have better performance data are not the ones who get the better salary.

In [ ]:
from pandas.plotting import scatter_matrix 
centers = pss.groupby("cluster").mean().reset_index
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 10
colors = np.array(['red','green','blue','yellow'])
plt.scatter(pss["SALARY"],pss["WINS_RPM"],c=colors[pss["cluster"]])
plt.scatter(centers.MP, centers.WINS_RPM,linewidths=50,marker='+',s=3,c='black')
plt.xlabel("SALARY")
plt.ylabel("WINS_RPM")

In [ ]:
scatter_matrix(pss[['SALARY','GP','MPG','ORPM','DRPM','RPM','WINS_RPM']],s=50,alpha=1,c=colors[ps["cluster"]],figsize=(20,20))

the clustering of "SALARY" and other features are much less clear than the last clustering we just made.Maybe that's because:
1. "SALARY" is decided by many featrues, 2D pichture can't describe or cluster the data point well.
2. "SALARY" is the label of this dataset which is not suitable for K-Means Unsupervised Data Analysis.

**SOME QUESTIONS**
1. What's the advantages if I do the PCA before clustering with K-Means.
2. What benefits can clustering the data bring to me when I do the machine learning.(Can I use it to train my model better or use it to select features efficiently?) 
3. How can I make a clustering picture with higher dimension(contains more features).The picture in this notebook are all 2d(contain 2 features).